In [15]:
results = {}

In [42]:
import pandas as pd
import numpy as np
import requests
import io
from tqdm import tqdm

# 딥러닝 라이브러리 TensorFlow/Keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MultiLabelBinarizer

# 시퀀스 길이 (과거 몇 회차의 데이터를 보고 다음을 예측할 것인가)
SEQUENCE_LENGTH = 10


def get_lotto_history(round, file_path, real_data, verbose=0, row_begin=0, row_end=30):
    try:
        # 인덱스를 지정해 시트 설정
        df = pd.read_excel(file_path, engine='openpyxl')
        # 필요한 컬럼 선택 및 이름 변경
        # lotto_df = df.iloc[row_begin:row_end, [0] + list(range(0, 7))]
        lotto_df = df[df.columns[1:7]]
        lotto_df.columns = ['1', '2', '3', '4', '5', '6']
        if verbose > 0:
            print(f"최신 데이터 로드 완료. [{round}]")
        return lotto_df.values.tolist()
    except requests.exceptions.RequestException as e:
        print(f"데이터를 가져오는 데 실패했습니다: {e}")
        return None


def create_sequences(data, seq_length):
    """
    데이터를 LSTM 학습에 맞는 시퀀스 형태로 변환합니다.
    """
    xs = []
    ys = []
    # Multi-hot 인코딩을 위해 1~45번까지의 클래스를 명시적으로 지정
    mlb = MultiLabelBinarizer(classes=list(range(1, 46)))
    
    # 전체 데이터를 Multi-hot 벡터로 변환
    encoded_data = mlb.fit_transform(data)
    
    for i in tqdm(range(len(encoded_data) - seq_length), desc="시퀀스 데이터 생성 중"):
        # 입력 시퀀스 (X): i부터 i+seq_length-1 까지
        xs.append(encoded_data[i:(i + seq_length)])
        # 타겟 (y): i+seq_length
        ys.append(encoded_data[i + seq_length])
        
    return np.array(xs), np.array(ys), mlb


def build_lstm_model(seq_length, num_features, neural_num=128, verbose=0, activation='sigmoid'):
    """
    LSTM 모델을 생성합니다.
    """
    model = Sequential()
    # 입력층: LSTM 레이어, input_shape=(시퀀스 길이, 피처 개수)
    model.add(LSTM(neural_num, return_sequences=True, activation=activation, input_shape=(seq_length, num_features)))
    model.add(LSTM(neural_num, return_sequences=True, activation=activation))
    # model.add(LSTM(neural_num, return_sequences=True, activation=activation))
    # model.add(LSTM(neural_num, return_sequences=True, activation=activation))
    # model.add(LSTM(neural_num, return_sequences=True, activation=activation))
    # model.add(LSTM(neural_num, return_sequences=True, activation=activation))
    # model.add(LSTM(neural_num, return_sequences=True, activation=activation))
    # model.add(LSTM(neural_num, return_sequences=True, activation=activation))
    # model.add(LSTM(neural_num, return_sequences=True, activation=activation))
    # model.add(LSTM(neural_num, return_sequences=True, activation=activation))
    # model.add(LSTM(neural_num, return_sequences=True, activation=activation))
    # model.add(LSTM(neural_num, return_sequences=True, activation=activation))
    # model.add(LSTM(neural_num, return_sequences=True, activation=activation))
    # model.add(LSTM(neural_num, return_sequences=True, activation=activation))
    # model.add(LSTM(neural_num, return_sequences=True, activation=activation))
    model.add(Dropout(0.2))
    # 중간층: LSTM 레이어
    model.add(LSTM(128, return_sequences=False))
    model.add(Dropout(0.2))
    # 출력층으로 가기 전 Dense 레이어
    model.add(Dense(64, activation='relu'))
    # 출력층: 45개의 로또 번호에 대한 확률을 출력 (sigmoid는 다중 라벨 분류에 사용)
    model.add(Dense(num_features, activation='sigmoid'))
    
    # 모델 컴파일
    # binary_crossentropy는 각 번호가 나올지/안나올지를 독립적으로 판단하는 다중 라벨 문제에 적합
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    if verbose > 0:
        model.summary()
    return model


file_names = [
    (1174, 'xlsx/1173.xlsx', [8, 11, 14, 17, 36, 39]),
    (1175, 'xlsx/1174.xlsx', [3, 4, 6, 8, 32, 42]),
    (1176, 'xlsx/1175.xlsx', [0, 0, 0, 0, 0, 0])
    ]


for file_name in file_names:
    # --- 코드 실행 ---
    # 1. 역대 당첨 번호 데이터 가져오기
    lotto_history = get_lotto_history(*file_name, row_end=10)
    if lotto_history is not None:
        neural_num = 64
        # 2. 데이터 전처리 및 시퀀스 생성
        X, y, mlb = create_sequences(lotto_history, SEQUENCE_LENGTH)
    
        # 3. LSTM 모델 빌드
        # num_features는 로또 번호의 개수(45)
        model = build_lstm_model(SEQUENCE_LENGTH, X.shape[2], neural_num=neural_num)
    
        # 4. 모델 학습 (데이터가 많아 시간이 소요될 수 있습니다)
        print("\nLSTM 모델 학습을 시작합니다...")
        model.fit(X, y, epochs=20, batch_size=32, validation_split=0.1, verbose=0)
    
        # 5. 1176회 예측
        print(f"\{file_name[0]}회 예측을 시작합니다...")
    
        # 예측을 위해 가장 마지막 시퀀스를 입력으로 사용
        last_sequence_raw = lotto_history[-SEQUENCE_LENGTH:]
        last_sequence_encoded = mlb.transform(last_sequence_raw)
    
        # Keras 모델은 배치 입력을 기대하므로 차원을 추가 (samples, timesteps, features)
        input_for_prediction = np.expand_dims(last_sequence_encoded, axis=0)
    
        # 예측 실행 (결과는 각 번호가 나올 확률)
        predicted_probabilities = model.predict(input_for_prediction)[0]
    
        # 확률이 높은 순으로 6개 번호의 인덱스를 추출
        # argsort는 값을 정렬했을 때의 원래 인덱스를 반환
        top_6_indices = np.argsort(predicted_probabilities)[-6:]
    
        # 인덱스는 0부터 시작하므로 1을 더해 실제 로또 번호로 변환
        predicted_numbers = sorted([int(i + 1) for i in top_6_indices])
        if file_name[0] not in results:
            results[file_name[0]] = [predicted_numbers]
        else:
            results[file_name[0]].append(predicted_numbers)
    
        print("\n--- 최종 예측 결과 ---")
        print(f"LSTM 모델 기반 {file_name[0]}회 예측 번호: {predicted_numbers} {neural_num}, ")

시퀀스 데이터 생성 중: 100%|██████████| 563/563 [00:00<00:00, 679146.72it/s]
/home/swhors/jupyter-workspace/finance/venv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



LSTM 모델 학습을 시작합니다...
\1174회 예측을 시작합니다...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 372ms/step

--- 최종 예측 결과 ---
LSTM 모델 기반 1174회 예측 번호: [6, 12, 19, 21, 38, 45] 64, 


시퀀스 데이터 생성 중: 100%|██████████| 564/564 [00:00<00:00, 838860.80it/s]


LSTM 모델 학습을 시작합니다...



/home/swhors/jupyter-workspace/finance/venv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


\1175회 예측을 시작합니다...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 369ms/step

--- 최종 예측 결과 ---
LSTM 모델 기반 1175회 예측 번호: [6, 12, 18, 21, 33, 45] 64, 


시퀀스 데이터 생성 중: 100%|██████████| 565/565 [00:00<00:00, 826860.35it/s]


LSTM 모델 학습을 시작합니다...



/home/swhors/jupyter-workspace/finance/venv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


\1176회 예측을 시작합니다...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 371ms/step

--- 최종 예측 결과 ---
LSTM 모델 기반 1176회 예측 번호: [6, 12, 13, 33, 38, 45] 64, 


In [41]:
for result in results:
    print("라운드", result)
    for metric in results[result]:
        print(metric)

라운드 1174
[12, 18, 21, 30, 33, 45]
[12, 18, 21, 33, 34, 38]
[12, 15, 18, 21, 33, 34]
[6, 7, 33, 34, 38, 45]
[12, 13, 15, 33, 34, 45]
[11, 12, 21, 33, 38, 45]
[12, 18, 21, 33, 34, 38]
[12, 18, 33, 34, 38, 45]
[12, 21, 28, 33, 38, 45]
[12, 15, 18, 33, 34, 38]
[12, 15, 33, 34, 38, 45]
라운드 1175
[6, 12, 21, 33, 34, 38]
[11, 12, 21, 33, 38, 45]
[12, 19, 21, 33, 38, 45]
[7, 11, 12, 18, 33, 34]
[12, 18, 21, 33, 38, 45]
[6, 11, 12, 33, 34, 45]
[12, 19, 21, 33, 38, 45]
[3, 12, 18, 21, 33, 38]
[12, 15, 18, 33, 38, 45]
[3, 12, 13, 21, 33, 38]
[12, 18, 21, 33, 34, 38]
라운드 1176
[3, 6, 12, 21, 33, 38]
[6, 12, 15, 21, 33, 38]
[7, 11, 12, 21, 38, 45]
[12, 13, 33, 34, 38, 45]
[6, 12, 33, 34, 38, 45]
[12, 18, 21, 33, 38, 45]
[12, 13, 33, 34, 38, 45]
[12, 19, 21, 33, 34, 45]
[12, 13, 15, 21, 33, 45]
[12, 21, 33, 34, 38, 45]
[12, 18, 21, 34, 38, 45]


In [7]:

actual_numbers = [
    [1175, 3, 4, 6, 8, 32, 42],
    [1174, 8, 11, 14, 17, 36, 39],
    [1173, 1, 5, 18, 20, 30, 35],
    [1172, 7, 9, 24, 40, 42, 44],
    [1171, 3, 6, 7, 11, 12, 17],
    [1170, 3, 13, 28, 34, 38, 42],
    [1169, 5, 12, 24, 26, 39, 42],
    [1168, 9, 21, 24, 30, 33, 37],
    [1167, 8, 23, 31, 35, 39, 40],
]